In [ ]:
from meeko import MoleculePreparation
from vina import Vina
from rdkit import Chem
from meeko import PDBQTMolecule
from meeko import PDBQTWriterLegacy

In [ ]:
v = Vina(sf_name='vina', cpu=8)
v.set_receptor('Data/9f6a.pdbqt')
v.set_ligand_from_file('Data/lig-1.pdbqt')

docking_box = {"center": [136.733, 172.819, 99.189], "box_size": [11.69, 7.09, 7.60]}
v.compute_vina_maps(**docking_box)

v.dock(exhaustiveness=8, n_poses=5)

In [4]:
preparator = MoleculePreparation()

In [ ]:
test_lig = Chem.SDMolSupplier('Data/ligands.sdf')[0]
test_lig

In [85]:
test_lig = Chem.AddHs(test_lig)

In [ ]:
Chem.rdDistGeom.EmbedMolecule(test_lig)

In [ ]:
test_lig

In [88]:
mol_setups = preparator.prepare(test_lig)

In [ ]:
for setup in mol_setups:
    pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
    if is_ok:
        print(pdbqt_string, end = "")

In [91]:
ligands = Chem.SDMolSupplier('Data/ligands.sdf')

In [ ]:
len(ligands)

In [ ]:
for i, mol in enumerate(ligands):
    if mol is None:
        continue
    if i >= 100:
        break
    print(ligands[i].GetProp('_Name'))

In [ ]:
from rdkit.Chem import SDMolSupplier
 
def process_ligands_in_batches(input_sdf_file, batch_size):
    suppl = SDMolSupplier(input_sdf_file)
 
    batch = []
    for idx, mol in enumerate(suppl):
        if mol is not None:
            batch.append(mol)
 
        # Process and flush the batch when the batch size is reached
        if len(batch) == batch_size:
            process_batch(batch, idx // batch_size)
            batch.clear()  # Clear the batch after processing
 
    # Process any remaining ligands in the final batch
    if batch:
        process_batch(batch, idx // batch_size)
 
def process_batch(batch, batch_idx):
    # Convert batch to PDBQT or perform docking in a single step
    print(f"Processing batch {batch_idx + 1} with {len(batch)} ligands.")
    # Replace with actual conversion and docking logic
 
# Example usage
input_sdf_file = 'ligands.sdf'
batch_size = 100  # Adjust batch size for performance and memory
 
process_ligands_in_batches(input_sdf_file, batch_size)

In [11]:
import os

In [12]:
if not os.path.exists('Output'):
    os.makedirs('Output')

In [8]:
from openbabel import pybel
from openbabel import openbabel

In [ ]:
mol_iter = pybel.readfile(filename = 'Data/ligands_10.sdf', format = 'sdf')

In [ ]:
type(mol_iter)

In [ ]:
for mol in mol_iter:
    print(mol.molwt)

In [17]:
from openbabel import openbabel
from openbabel import pybel

# Input and output file paths
input_smi = "Data/ligands_10_2.smi"
output_sdf = "output_molecules4.pdbqt"

converter = openbabel.OBConversion()
converter.SetInAndOutFormats("sdf", "pdbqt")

# Desired pH
pH = 7.0

# Open the output file
with open(output_sdf, "w") as outfile:
    # Read molecules from the SDF file
    for mol in pybel.readfile("smi", input_smi):
        # Convert to OBMol for direct manipulation
        obmol = mol.OBMol

        # Adjust hydrogens at the specified pH
        obmol.AddHydrogens(False, True, pH)
    
        # Convert the adjusted molecule back to Pybel format
        adjusted_mol = pybel.Molecule(obmol)

        adjusted_mol.make3D()

        # Write output
        outfile.write(adjusted_mol.write("pdbqt"))
